In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from os.path import join as pjoin
from matplotlib import font_manager
font_manager.fontManager.addfont('.supportfiles/arial.ttf')

In [ ]:
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


In [ ]:
# define path
nod_path = 'PATH TO DATASET'
fig_path = 'PATH TO SAVE FIGURES' 

### imagenet task

In [ ]:

# subject list generation
subs = ['sub-{:02d}'.format(i) for i in range(1,31)]

# sub array
responserate = np.zeros((len(subs),), dtype=object)
accs = np.zeros((len(subs),), dtype=object)

# for loop
for i, sub in enumerate(subs):
    # initialize behavioral list
    all_run_response_rate = []
    all_run_accuracy = []
    sessions = sorted([_ for _ in os.listdir(pjoin(nod_path, sub)) if 'imagenet' in _])
    # for loop
    for session in sessions:
        # func path
        sub_func_path = pjoin(nod_path, sub, session, 'func')
        # events files
        events_files = sorted([_ for _ in os.listdir(sub_func_path) if 'events.tsv' in _])
        # for loop
        for events_file in events_files:

            # behavior data
            events = pd.read_csv(pjoin(sub_func_path, events_file), sep='\t')
            trial_type = (events['trial_type'].values -1).astype(np.int64)
            response = events['response'].values
            response_time = events['response_time'].values

            # animate label
            ssup_path = './supportfiles/'
            animacylabel = sio.loadmat(pjoin(ssup_path,'animate_or_not.mat'))['animate_label'].reshape(-1)

            # response rate
            run_resprate = 1 - np.sum(response==0)/len(response)

            # acc
            groundtrue = animacylabel[trial_type]
            # acc | response
            run_acc = np.mean(response[np.where(response!=0)] == groundtrue[np.where(response!=0)])
            # # acc counting no response as wrong
            # run_acc = np.mean(response == groundtrue)

            # append
            all_run_response_rate.append(run_resprate)
            all_run_accuracy.append(run_acc)
    # gather each subject performance
    responserate[i] = np.array(all_run_response_rate)
    accs[i] = np.array(all_run_accuracy)

In [ ]:
# plots settings
mpl.rcParams.update({'font.size': 10.5, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})

fig,ax = plt.subplots(figsize=(7,1))
width=0.7
font_size = 10.5

# index apponitment
draw_index = accs # responserate #
index_name = 'Accuracy' #'Response Rate' # 

# draw for each subject
for i, data in enumerate(draw_index) :
    parts = ax.violinplot(data, positions=[i], widths=width,showextrema=False,showmedians=False, vert=True)
    for pc in parts['bodies']:
        pc.set_facecolor('#81C6E8')
        pc.set_alpha(0.65)
    q1, medians, q3 = np.percentile(data, [25, 50, 75], axis=0)
    whiskers = np.array([adjacent_values(np.sort(data), q1, q3)])
    whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]
    ax.scatter(i, medians, marker='o', color='#B73E3E', s=10, zorder=3)
    ax.vlines(i, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1.2, alpha=0.3)
    ax.vlines(i, q1, q3, color='red',edgecolors='#E97777', linestyle='-', lw=3.6)


ax.set_xlim([-0.9,29.52])
ax.set_xticks(np.arange(30))
ax.set_xticklabels(['{:d}'.format(i+1) for i in np.arange(30)])
ax.set_xlabel('subject',fontsize=font_size)

if index_name == 'Accuracy':
    ax.set_ylim([0.58,1.01])
    ax.set_yticks([0.7,1.0]) # accs
if index_name == 'Response Rate':
    ax.set_ylim([0.35,1.01])
    ax.set_yticks([0.5,1.0]) 
# ax.set_yticklabels([],fontsize=10.5)
ax.set_ylabel(index_name, fontsize=font_size)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(2.0)
ax.spines['bottom'].set_linewidth(2.0)
ax.tick_params(labelsize=font_size, width=1, length=3, direction="in", pad=2)

# save out
fig.savefig(pjoin(fig_path, f'{index_name}-IM.jpg'),dpi=72*4,bbox_inches='tight')
plt.show()

In [ ]:
# subject list generation
subs = ['sub-{:02d}'.format(i) for i in range(1,10)]

# sub results array
rts = np.zeros((len(subs),), dtype=object)
coco_accs = np.zeros((len(subs),), dtype=object)

# for loop
for i, sub in enumerate(subs):
    # initialize behavioral list
    all_run_rt = []
    all_run_accuracy = []
    session = 'ses-coco'
    sub_func_path = pjoin(nod_path, sub, session, 'func')
    events_files = sorted([_ for _ in os.listdir(sub_func_path) if 'events.tsv' in _])
    # for loop
    for events_file in events_files:

        # behavior data
        events = pd.read_csv(pjoin(sub_func_path, events_file), sep='\t')
        trial_type = events['trial_type'].values
        response = events['response'].values
        response_time = events['response_time'].values   

        # acc
        run_acc = np.mean(response[np.where(trial_type==2000)])
        run_rt = np.mean(response_time[np.where(trial_type==2000)])
        # append
        all_run_accuracy.append(run_acc)
        all_run_rt.append(run_rt)
    # gather for each subject
    rts[i] = np.array(all_run_rt)
    coco_accs[i] = np.array(all_run_accuracy)


### draw for coco

In [ ]:
#
mpl.rcParams.update({'font.size': 10.5, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})
fig,axs = plt.subplots(1,3,figsize=(7,1))
width=0.7
font_size = 10.5

# indices wait to draw
draw_indices =  [coco_accs] #1000*rts #
index_name = 'Accuracy' # 'Response Time' #

for i, ax in enumerate(axs):
    
    draw_index = draw_indices[i]
    for i, data in enumerate(draw_index) :
        parts = ax.violinplot(data, positions=[i], widths=width,showextrema=False,showmedians=False, vert=True)
        for pc in parts['bodies']:
            pc.set_facecolor('#81C6E8')
            pc.set_alpha(0.85)
        q1, medians, q3 = np.percentile(data, [25, 50, 75], axis=0)
        whiskers = np.array([adjacent_values(np.sort(data), q1, q3)])
        whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]
        ax.scatter(i, medians, marker='o', color='#B73E3E', s=10, zorder=3)
        ax.vlines(i, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1.2, alpha=0.3)
        ax.vlines(i, q1, q3, color='red',edgecolors='#E97777', linestyle='-', lw=3.6)
        
    ax.set_xlim([-0.8,8.5])
    # ax.set_yticks([0,1,2,3])
    # ax.set_yticklabels([],fontsize=10.5)
    ax.set_ylim([0.01,1.01])
    ax.set_xticks(np.arange(9))
    ax.set_xticklabels(['{:d}'.format(i+1) for i in np.arange(9)])
    # ax.set_ylabel(index_name, fontsize=font_size)
    ax.set_xlabel('subject',fontsize=font_size)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2.0)
    ax.spines['bottom'].set_linewidth(2.0)
    ax.tick_params(labelsize=font_size, width=1, length=3, direction="in", pad=2)
    # ax.set_tick
    # ax.plot([0.5,0.5], [-2,4], ls='--',color='#DD5353',lw=2)
    # ax.set_ylim([0.5,3.5])
fig.savefig(pjoin(fig_path, f'{index_name}-coco&floc&retino.jpg'),dpi=72*4,bbox_inches='tight')
plt.show()